### Loading data and a model

In [11]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(
X_train_full, y_train_full)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

model = keras.models.Sequential([
keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
keras.layers.Dense(1)
])
model.compile(loss="mean_squared_error", optimizer="sgd")
history = model.fit(X_train, y_train, epochs=20,
validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3] # pretend these are new instances
y_pred = model.predict(X_new)

Epoch 1/20
155/363 [===========>..................] - ETA: 0s - loss: 1.0312 

2022-08-18 08:56:08.357026: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


363/363 [==============================] - 0s 552us/step - loss: 0.7446 - val_loss: 0.6632
Epoch 2/20
363/363 [==============================] - 0s 401us/step - loss: 0.4530 - val_loss: 0.7184
Epoch 3/20
363/363 [==============================] - 0s 392us/step - loss: 0.4254 - val_loss: 0.4079
Epoch 4/20
363/363 [==============================] - 0s 390us/step - loss: 0.4066 - val_loss: 0.4682
Epoch 5/20
363/363 [==============================] - 0s 509us/step - loss: 0.4113 - val_loss: 0.3969
Epoch 6/20
363/363 [==============================] - 0s 392us/step - loss: 0.3951 - val_loss: 0.4281
Epoch 7/20
363/363 [==============================] - 0s 391us/step - loss: 0.4000 - val_loss: 0.5870
Epoch 8/20
363/363 [==============================] - 0s 387us/step - loss: 0.3945 - val_loss: 1.0952
Epoch 9/20
363/363 [==============================] - 0s 396us/step - loss: 0.3845 - val_loss: 0.4156
Epoch 10/20
363/363 [==============================] - 0s 455us/step - loss: 0.3836 - val_los

In [7]:
import os
root_logdir = os.path.join(os.curdir, "my_logs")
root_logdir

'./my_logs'

In [3]:
import time
def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

In [9]:
run_logdir = get_run_logdir()
run_logdir

'./my_logs/run_2022_08_18-08_54_41'

In [16]:
tensorboard_cb = keras.callbacks.TensorBoard(get_run_logdir())
history = model.fit(
    X_train, 
    y_train, 
    epochs=30,
    validation_data=(X_valid, y_valid),
    callbacks=[tensorboard_cb]
)

Epoch 1/30
363/363 [==============================] - 0s 469us/step - loss: 0.3074 - val_loss: 0.4308
Epoch 2/30
363/363 [==============================] - 0s 397us/step - loss: 0.3079 - val_loss: 0.3433
Epoch 3/30
363/363 [==============================] - 0s 398us/step - loss: 0.3070 - val_loss: 0.3339
Epoch 4/30
363/363 [==============================] - 0s 567us/step - loss: 0.3081 - val_loss: 0.3424
Epoch 5/30
363/363 [==============================] - 0s 1ms/step - loss: 0.3053 - val_loss: 0.3357
Epoch 6/30
363/363 [==============================] - 0s 1ms/step - loss: 0.3050 - val_loss: 0.3596
Epoch 7/30
363/363 [==============================] - 0s 917us/step - loss: 0.3058 - val_loss: 0.3091
Epoch 8/30
363/363 [==============================] - 0s 497us/step - loss: 0.3054 - val_loss: 0.3075
Epoch 9/30
363/363 [==============================] - 0s 402us/step - loss: 0.3066 - val_loss: 0.3624
Epoch 10/30
363/363 [==============================] - 0s 405us/step - loss: 0.3043 - 

In [21]:
# Create a personalized log
import tensorflow as tf
import numpy as np

test_logdir = get_run_logdir()
writer = tf.summary.create_file_writer(test_logdir)
with writer.as_default():
    for step in range(1, 1000 + 1):
        tf.summary.scalar("my_scalar", np.sin(step / 10), step=step)
        data = (np.random.randn(100) + 2) * step / 100 # some random data
        tf.summary.histogram("my_hist", data, buckets=50, step=step)
        images = np.random.rand(2, 32, 32, 3) # random 32×32 RGB images
        tf.summary.image("my_images", images * step / 1000, step=step)
        texts = ["The step is " + str(step), "Its square is " + str(step**2)]
        tf.summary.text("my_text", texts, step=step)
        sine_wave = tf.math.sin(tf.range(12000) / 48000 * 2 * np.pi * step)
        audio = tf.reshape(tf.cast(sine_wave, tf.float32), [1, -1, 1])
        tf.summary.audio("my_audio", audio, sample_rate=48000, step=step)